CS524: Introduction to Optimization Lecture 6
======================================

## Michael Ferris<br> Computer Sciences Department <br> University of Wisconsin-Madison

## September 16, 2024
--------------

In [1]:
import numpy as np
import pandas as pd
import sys
from gamspy import (
    Container,Set,Alias,Parameter,Variable,Equation,Model,Problem,Sense,Options,
    Domain,Number,Sum,Product,Smax,Smin,Ord,Card,SpecialValues,
)
options = Options(variable_listing_limit=0, equation_listing_limit=50)

m = Container(options=options)
T = Set(m,'T',records=['Jan', 'Feb', 'Mar', 'Apr'])
I0 = Parameter(m,'I0',description='Initial Inventory',records=500)
W0 = Parameter(m,'W0',description='Initial Number of Workers',records=100)
alpha = Parameter(m,'alpha',description='Dollars per month for worker',records=1500)
beta = Parameter(m,'beta',description='Dollars per hour for overtime',records=13)
MAXH = Parameter(m,'MAXH',description='max hours per month',records=160)
MAXO = Parameter(m,'MAXO',description='max overtime hours per month',records=20)
a = Parameter(m,'a',description='labor hours per shoe',records=4)
delta = Parameter(m,'delta',description='Raw material costs (dollars) per shoe',records=15)
eta = Parameter(m,'eta',description='hiring cost (dollars) per worker',records=1600)
zeta = Parameter(m,'zeta',description='firing cost (dollars) per worker',records=2000)
iota = Parameter(m,'iota',description='inventory cost (dollars) per shoe',records=3)
d = Parameter(m,'d',[T],description='Demand',records=np.array([ 3000., 5000., 2000., 1000. ]))

x = Variable(m,'x','positive',[T],description='Production in period T')
I = Variable(m,'I','positive',[T],description='Ending inventory in period T')
w = Variable(m,'w','positive',[T],description='Worker level at end of period T')
h = Variable(m,'h','positive',[T],description='Number of workers hired at beginning of T')
f = Variable(m,'f','positive',[T],description='Number of workers fired at beginning of T')
o = Variable(m,'o','positive',[T],description='Number of overtime hours in period T')

In [2]:
# Requirements constraints
RegLabor_eq = Equation(m,'RegLabor_eq',domain=[T])
RegLabor_eq[T]= a*x[T] <= MAXH * w[T] + o[T]

OverLabor_eq = Equation(m,'OverLabor_eq',domain=[T])
OverLabor_eq[T]= o[T] <= MAXO * w[T]

BalShoe_eq = Equation(m,'BalShoe_eq',domain=[T])
BalShoe_eq[T]= I[T] == I0.where[T.first] + I[T.lag(1)] + x[T] - d[T]

BalPeople_eq = Equation(m,'BalPeople_eq',domain=[T])
BalPeople_eq[T]= w[T] == W0.where[T.first] + w[T.lag(1)] + h[T] - f[T]

shoeco = Model(m,"shoeco",
    equations=m.getEquations(),
    problem=Problem.LP,
    sense=Sense.MIN,
    objective=Sum(T, delta*x[T] + alpha * w[T] + beta * o[T] + eta * h[T] +
                    zeta * f[T] + iota * I[T])
)

shoeco.solve()

,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,OptimalGlobal,692500,17,25,LP,CPLEX,0


In [3]:
print(shoeco.getEquationListing())
objective = {}
objective['shoeco'] = shoeco.objective_value
results = Parameter(m,'results',domain=[T,'*'])
results[T,'x'] = x.l[T]
results[T,'I'] = I.l[T]
results[T,'w'] = w.l[T]
results[T,'h'] = h.l[T]
results[T,'f'] = f.l[T]
results[T,'o'] = o.l[T]
display(m['results'].pivot())

RegLabor_eq(Jan)..  4*x(Jan) - 160*w(Jan) - o(Jan) =L= 0 ; (LHS = 0)
RegLabor_eq(Feb)..  4*x(Feb) - 160*w(Feb) - o(Feb) =L= 0 ; (LHS = 0)
RegLabor_eq(Mar)..  4*x(Mar) - 160*w(Mar) - o(Mar) =L= 0 ; (LHS = 0)
RegLabor_eq(Apr)..  4*x(Apr) - 160*w(Apr) - o(Apr) =L= 0 ; (LHS = 0)
OverLabor_eq(Jan)..  - 20*w(Jan) + o(Jan) =L= 0 ; (LHS = 0)
OverLabor_eq(Feb)..  - 20*w(Feb) + o(Feb) =L= 0 ; (LHS = 0)
OverLabor_eq(Mar)..  - 20*w(Mar) + o(Mar) =L= 0 ; (LHS = 0)
OverLabor_eq(Apr)..  - 20*w(Apr) + o(Apr) =L= 0 ; (LHS = 0)
BalShoe_eq(Jan)..  - x(Jan) + I(Jan) =E= -2500 ; (LHS = 0, INFES = 2500 ****)
BalShoe_eq(Feb)..  - x(Feb) - I(Jan) + I(Feb) =E= -5000 ; (LHS = 0, INFES = 5000 ****)
BalShoe_eq(Mar)..  - x(Mar) - I(Feb) + I(Mar) =E= -2000 ; (LHS = 0, INFES = 2000 ****)
BalShoe_eq(Apr)..  - x(Apr) - I(Mar) + I(Apr) =E= -1000 ; (LHS = 0, INFES = 1000 ****)
BalPeople_eq(Jan)..  w(Jan) - h(Jan) + f(Jan) =E= 100 ; (LHS = 0, INFES = 100 ****)
BalPeople_eq(Feb)..  - w(Jan) + w(Feb) - h(Feb) + f(Feb) =E= 

,x,I,w,f
Jan,3750.0,1250.0,93.75,6.25
Feb,3750.0,0.0,93.75,0.00
Mar,2000.0,0.0,50.00,43.75
Apr,1000.0,0.0,50.00,0.00


Explain why you don't fire 25 people in April (only need 25 people to generate 1000 shoes)

Now lets model the case where backlogging is allowed

In [4]:
L = Variable(m,'L','positive',[T],description='Leftover inventory in scenario')
S = Variable(m,'S','positive',[T],description='Shortage inventory in scenario')

theta = Parameter(m,'theta',description='Backlog cost',records=20)

BacklogDef_eq = Equation(m,'BacklogDef_eq',domain=[T])
BacklogDef_eq[T]=  I[T] == L[T] - S[T]

shoeco = Model(m,"shoeco",
    equations=m.getEquations(),
    problem=Problem.LP,
    sense=Sense.MIN,
    objective=Sum(T, delta*x[T] + alpha * w[T] + beta * o[T] + eta * h[T] +
                    zeta * f[T] + iota * L[T] + theta * S[T] )
)

I.lo[T] = SpecialValues.NEGINF
I.lo[T].where[T.last] = 0

shoeco.solve()

,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,OptimalGlobal,690000,21,33,LP,CPLEX,0


In [5]:
objective['backlog'] = shoeco.objective_value
results[T,'B x'] = x.l[T]
results[T,'B I'] = I.l[T]
results[T,'B w'] = w.l[T]
results[T,'B h'] = h.l[T]
results[T,'B f'] = f.l[T]
results[T,'B o'] = o.l[T]
results[T,'B L'] = L.l[T]
results[T,'B S'] = S.l[T]

display(m['results'].pivot())

,x,I,w,f,B x,B I,B w,B f,B L,B S
Jan,3750.0,1250.0,93.75,6.25,3750.0,1250.0,93.75,6.25,1250.0,0.0
Feb,3750.0,0.0,93.75,0.00,3750.0,0.0,93.75,0.00,0.0,0.0
Mar,2000.0,0.0,50.00,43.75,1500.0,-500.0,37.50,56.25,0.0,500.0
Apr,1000.0,0.0,50.00,0.00,1500.0,0.0,37.50,0.00,0.0,0.0


Now suppose we have demand scenarios S (in this example 3).
Let's see how easy it is to do scenarios and minimax.

In [6]:
Scen = Set(m,'Scen',records=[f"s{i}" for i in range(1, 4)])
# Scen = Set(m,'Scen',records=range(1, 4))
dU = Parameter(m,'dU',[T,Scen],records=np.array([
    [ 3000., 5000., 1000. ],
    [ 5000.,  3000.,  1000. ],
    [ 2000.,  1000.,  5000. ],
    [ 1000.,  2000.,  4000. ]]))
LU = Variable(m,'LU','positive',[T,Scen],description='Leftover inventory in scenario')
SU = Variable(m,'SU','positive',[T,Scen],description='Shortage inventory in scenario')
IU = Variable(m,'IU','free',[T,Scen],description='Inventory in scenario')
z = Variable(m,'z','free',description='Auxiliary var for maximum')

zdef_eq = Equation(m,'zdef_eq',domain=[Scen])
zdef_eq[Scen]= (
    z >= Sum(T, delta*x[T] + alpha * w[T] + beta * o[T] + eta * h[T] +
        zeta * f[T]  + iota * LU[T,Scen] + theta * SU[T,Scen]))
    
BacklogDefU_eq = Equation(m,'BacklogDefU_eq',domain=[T,Scen])
BacklogDefU_eq[T,Scen]=  IU[T,Scen] == LU[T,Scen] - SU[T,Scen]

BalShoeU_eq = Equation(m,'BalShoeU_eq',domain=[T,Scen])
BalShoeU_eq[T,Scen]=  IU[T,Scen] == I0.where[T.first] + IU[T.lag(1),Scen] + x[T] - dU[T,Scen]

minimax = Model(m,"minimax",
    equations=[RegLabor_eq, OverLabor_eq, BalShoeU_eq, BalPeople_eq, BacklogDefU_eq, zdef_eq],
    problem=Problem.LP,
    sense=Sense.MIN,
    objective=z
)

IU.lo["Apr",Scen] = 0

minimax.solve()

,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,OptimalGlobal,692539.855072464,39,57,LP,CPLEX,0


In [7]:
objective['minimax'] = minimax.objective_value
display(objective)

{'shoeco': 692500.0, 'backlog': 690000.0, 'minimax': 692539.8550724636}